<a href="https://colab.research.google.com/github/sahilsait/synthetic-EHR-data-generation/blob/main/CS271_final_project_preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd

patients = pd.read_csv('/content/drive/MyDrive/datasets/csv/patients.csv', usecols=["Id", "BIRTHDATE", "RACE", "ETHNICITY", "MARITAL", "GENDER"])
conditions = pd.read_csv('/content/drive/MyDrive/datasets/csv/conditions.csv', usecols=["PATIENT", "DESCRIPTION"])
encounters = pd.read_csv('/content/drive/MyDrive/datasets/csv/encounters.csv', usecols=["START", "PATIENT", "ENCOUNTERCLASS"])
observations = pd.read_csv('/content/drive/MyDrive/datasets/csv/observations.csv', usecols=["PATIENT", "DESCRIPTION", "VALUE", "UNITS"])

In [ ]:
from datetime import datetime

patients = patients.rename(columns={"Id": "PATIENT"})
patients["BIRTHDATE"] = pd.to_datetime(patients["BIRTHDATE"])
patients["age"] = ((pd.Timestamp("now") - patients["BIRTHDATE"]).dt.days / 365.25).astype(int)
patients = patients.drop(columns=["BIRTHDATE"])
patients["MARITAL"] = patients["MARITAL"].fillna("Unknown")

In [ ]:
encounters["START"] = pd.to_datetime(encounters["START"])
encounter_counts = encounters.groupby("PATIENT").size().reset_index(name="num_encounters")

# Get most recent encounter type
last_encounter = encounters.sort_values(by=["PATIENT", "START"]).groupby("PATIENT").last().reset_index()
last_encounter = last_encounter[["PATIENT", "ENCOUNTERCLASS"]].rename(columns={"ENCOUNTERCLASS": "last_encounter_type"})

encounter_features = encounter_counts.merge(last_encounter, on="PATIENT", how="left")

In [ ]:
target_vitals = {
    "Body Mass Index": "kg/m2",
    "Systolic Blood Pressure": "mm[Hg]",
    "Heart rate": "/min"
}

# Filter only valid unit rows
obs_filtered = observations[
    observations.apply(lambda row: row["DESCRIPTION"] in target_vitals and row["UNITS"] == target_vitals[row["DESCRIPTION"]], axis=1)
]

# Get most recent value for each (patient, DESCRIPTION)
latest_obs = (
    obs_filtered.sort_values(["PATIENT", "DESCRIPTION"])
    .drop_duplicates(subset=["PATIENT", "DESCRIPTION"], keep="last")
)

vital_pivot = latest_obs.pivot(index="PATIENT", columns="DESCRIPTION", values="VALUE").reset_index()

In [ ]:
obs_filtered.value_counts("DESCRIPTION")

,count
DESCRIPTION,
Systolic Blood Pressure,12963
Heart rate,12552
Body Mass Index,11451


In [ ]:
print(len(patients))
print(len(conditions))
print(len(observations))
print(len(encounters))

1171
8376
299697
53346


In [ ]:
df = patients.merge(conditions, on="PATIENT", how="left")
df = df.merge(encounter_features, on="PATIENT", how="left")
df = df.merge(vital_pivot, on="PATIENT", how="left")

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8395 entries, 0 to 8394
Data columns (total 12 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   PATIENT                  8395 non-null   object
 1   MARITAL                  8395 non-null   object
 2   RACE                     8395 non-null   object
 3   ETHNICITY                8395 non-null   object
 4   GENDER                   8395 non-null   object
 5   age                      8395 non-null   int64 
 6   DESCRIPTION              8376 non-null   object
 7   num_encounters           8395 non-null   int64 
 8   last_encounter_type      8395 non-null   object
 9   Body Mass Index          8327 non-null   object
 10  Heart rate               8395 non-null   object
 11  Systolic Blood Pressure  8395 non-null   object
dtypes: int64(2), object(10)
memory usage: 787.2+ KB


In [ ]:
df = df.rename(columns={"Body Mass Index": "body_mass_index", "Heart rate": "heart_rate", "Systolic Blood Pressure": "systolic_blood_pressure"})

In [ ]:
df = df.dropna(subset=["DESCRIPTION", "body_mass_index"], how="any")

In [ ]:
cols = [shape of X: (8322, 152)]
df[cols] = df[cols].apply(pd.to_numeric, errors="coerce")
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8322 entries, 0 to 8394
Data columns (total 12 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   PATIENT                  8322 non-null   object 
 1   MARITAL                  8322 non-null   object 
 2   RACE                     8322 non-null   object 
 3   ETHNICITY                8322 non-null   object 
 4   GENDER                   8322 non-null   object 
 5   age                      8322 non-null   int64  
 6   DESCRIPTION              8322 non-null   object 
 7   num_encounters           8322 non-null   int64  
 8   last_encounter_type      8322 non-null   object 
 9   body_mass_index          8322 non-null   float64
 10  heart_rate               8322 non-null   float64
 11  systolic_blood_pressure  8322 non-null   float64
dtypes: float64(3), int64(2), object(7)
memory usage: 845.2+ KB


<ipython-input-13-48d65b33920a>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[cols] = df[cols].apply(pd.to_numeric, errors="coerce")


In [ ]:
df = df.drop(columns=['PATIENT'])
df.to_csv("/content/drive/MyDrive/datasets/ehr_clean.csv", index=False)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8322 entries, 0 to 8394
Data columns (total 11 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   MARITAL                  8322 non-null   object 
 1   RACE                     8322 non-null   object 
 2   ETHNICITY                8322 non-null   object 
 3   GENDER                   8322 non-null   object 
 4   age                      8322 non-null   int64  
 5   DESCRIPTION              8322 non-null   object 
 6   num_encounters           8322 non-null   int64  
 7   last_encounter_type      8322 non-null   object 
 8   body_mass_index          8322 non-null   float64
 9   heart_rate               8322 non-null   float64
 10  systolic_blood_pressure  8322 non-null   float64
dtypes: float64(3), int64(2), object(6)
memory usage: 780.2+ KB
